In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
import csv
import gzip
import itertools
import pandas as pd
import numpy as np

Here we read all the sequence indeces for the forward and reverse reads

In [2]:
forward_dict = SeqIO.to_dict(SeqIO.parse("FW_primers.fa", "fasta"))
forward_dict = {k:str(v.seq) for k, v in forward_dict.items()}
reverse_dict = SeqIO.to_dict(SeqIO.parse("RV_primers.fa", "fasta"))
reverse_dict = {k:str(v.seq.reverse_complement()) for k, v in reverse_dict.items()}

Now, we read all the plasmid sequences

In [3]:
plasmids = {}
with open('Plasmid_ID_correct_position_jake_rcbc100.csv', 'r') as csvfile:
    table_reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in table_reader:
        plasmids[row["Barcode name"]]= row["ID plasmid sequence"]

In [4]:
table_reader

Let's iterate the two sequence files and produce the table with the distances

In [6]:
def find_hash_position(sequence, tags):
    #print("________")
    #print(sequence)
    #print("________")
    for key, value in tags.items():
    #    print(value)
        position = sequence.find(value)
        if position != -1:
            return (key, position)
    return ("none", -1)

fw_f = "../Jake13.merged.fastq.gz"
distances_f = "distances_merge.txt"
with gzip.open(fw_f, "rt") as r1,open(distances_f,"w") as f3:
    f3.write("fw_name,fw_pos,rev_name,rev_pos,pl_name,pl_pos\n")
    for fw in SeqIO.parse(r1, "fastq") :
        #print(fw)
        str_seq = str(fw.seq)
        fw_name, fw_pos = find_hash_position(str_seq, forward_dict)
        rv_name, rv_pos = find_hash_position(str_seq, reverse_dict)
        pl_name, pl_pos = find_hash_position(str_seq, plasmids)
        f3.write(",".join([fw_name, str(fw_pos), rv_name, str(rv_pos), pl_name,str(pl_pos)])) 
        f3.write("\n")
    
    

In [7]:
df = pd.read_csv(distances_f)
df

,fw_name,fw_pos,rev_name,rev_pos,pl_name,pl_pos
0,7_F,0,B_R,54,rbc106,42
1,5_F,0,B_R,55,rbc106,43
2,9_F,0,B_R,56,rbc108,44
3,6_F,0,B_R,56,rbc106,44
4,5_F,0,B_R,55,rbc106,43
...,...,...,...,...,...,...
1452,5_F,0,B_R,55,rbc106,43
1453,7_F,0,B_R,54,rbc106,42
1454,7_F,0,B_R,54,rbc106,42
1455,5_F,0,B_R,55,rbc109,43


In [8]:
summ=df[['fw_name', 'rev_name', 'pl_name', 'pl_pos']].groupby(['fw_name', 'rev_name', 'pl_name']).agg( ['count','mean'])

In [9]:
summ.to_csv("Jake05_S101_L001_R_001.csv", sep=',')

In [10]:
summ

pl_pos           
                          count       mean
fw_name rev_name pl_name                  
10_F    B_R      none         1  -1.000000
        C_R      rbc106       1  42.000000
        D_R      rbc108       1  42.000000
        E_R      rbc105       1  42.000000
                 rbc106       1  42.000000
11_F    D_R      rbc101       1  43.000000
                 rbc106       1  43.000000
                 rbc109       1  43.000000
5_F     B_R      none        11  -1.000000
                 rbc101       1  43.000000
                 rbc105      21  43.000000
                 rbc106     424  43.000000
                 rbc107      21  43.000000
                 rbc108       9  43.000000
                 rbc109      11  43.000000
        F_R      rbc106       2  43.000000
                 rbc107       2  43.000000
        none     rbc105       1  43.000000
                 rbc106       6  43.000000
                 rbc107       4  43.000000
6_F     B_R      none         1  -1.000000
                 rbc101       1  44.000000
                 rbc105      55  44.000000
                 rbc106     190  44.000000
                 rbc107       3  44.000000
                 rbc108       1  44.000000
        D_R      rbc104       1  44.000000
        F_R      rbc106       1  44.000000
                 rbc107       1  44.000000
        none     rbc105       2  44.000000
                 rbc106       4  44.000000
7_F     B_R      none         5  -1.000000
                 rbc105      12  42.000000
                 rbc106     253  42.000000
                 rbc107       7  42.000000
        D_R      none         1  -1.000000
                 rbc104       4  42.000000
        F_R      rbc106       1  42.000000
        none     rbc105       1  42.000000
                 rbc106      12  42.000000
8_F     B_R      none         1  -1.000000
                 rbc105       5  43.000000
                 rbc106     151  43.000000
                 rbc107       3  43.000000
                 rbc109       3  43.000000
        none     rbc106       2  43.000000
9_F     B_R      none         2  -1.000000
                 rbc105      17  44.000000
                 rbc106     141  44.000000
                 rbc107       6  44.000000
                 rbc108       3  44.000000
        F_R      rbc106       2  44.000000
        none     rbc106       8  44.000000
none    B_R      none         1  -1.000000
                 rbc105       3  43.333333
                 rbc106      22  42.909091
        F_R      rbc106       1  44.000000
        none     none         7  -1.000000
                 rbc106       2  43.000000